# Importing Libraries

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from bs4 import BeautifulSoup
import requests

import os
import sys
sys.path.append("../source/")
sys.path.append("../")

import paths

# Prototyping a Single-Shot Scraper

In [3]:
Response = requests.get("https://www.rba.gov.au/monetary-policy/rba-board-minutes/2023/")

In [4]:
Soup = BeautifulSoup(Response.text,"html.parser")
#Soup

In [5]:
UL = Soup.find("ul", {"class": "list-articles"})

In [6]:
As = UL.find_all("a")
#As

In [7]:
Links = [{"Link" : x["href"], "Title" : x.text.replace(" ", "_")} for x in As]
Links

[{'Link': '/monetary-policy/rba-board-minutes/2023/2023-09-05.html',
  'Title': '5_September_2023'},
 {'Link': '/monetary-policy/rba-board-minutes/2023/2023-08-01.html',
  'Title': '1_August_2023'},
 {'Link': '/monetary-policy/rba-board-minutes/2023/2023-07-04.html',
  'Title': '4_July_2023'},
 {'Link': '/monetary-policy/rba-board-minutes/2023/2023-06-06.html',
  'Title': '6_June_2023'},
 {'Link': '/monetary-policy/rba-board-minutes/2023/2023-05-02.html',
  'Title': '2_May_2023'},
 {'Link': '/monetary-policy/rba-board-minutes/2023/2023-04-04.html',
  'Title': '4_April_2023'},
 {'Link': '/monetary-policy/rba-board-minutes/2023/2023-03-07.html',
  'Title': '7_March_2023'},
 {'Link': '/monetary-policy/rba-board-minutes/2023/2023-02-07.html',
  'Title': '7_February_2023'}]

In [8]:
def ETL_Data(Links):
    
    for x in Links:
        RBA_Base_Link = "https://www.rba.gov.au/"

        #Getting Response for Given Link
        Response = requests.get(RBA_Base_Link + x["Link"])
        Soup = BeautifulSoup(Response.text,"html.parser")

        #Finding Content Div
        Div = Soup.find("div", {"id": "content"})

        T = []
        
        for y in Div:
            T.append(y.text)
    
        #Cleaning Text
        Text = [text.replace("\n", "").replace("\t", "").replace("\xa0", " ").replace(",", ", ").replace("  ", " ") for text in T]
        Text = [t for t in Text if t != ""] 

        #Dumping to Disk if not there Already
        File_Path = paths.TRANSFORMED_DATA_DIR / f'{x["Title"]}.txt'
        
        if not File_Path.exists():
            with open (File_Path, "w") as f:
                f.write('\n'.join(Text))

# Scraping Data From Start_Year to End_Year

In [9]:
Start_Year = 2019
End_Year = 2023

In [10]:
RBA_Base_Link = "https://www.rba.gov.au/"
Monetary_Policy_Link = "monetary-policy/rba-board-minutes/"

In [11]:
#For Each Year in the Rande Start_Year-End_Year we are going to Get all the Links for the Policy Minutes and then we fetch Data for Each Link

while Start_Year <= End_Year:
    
    Complete_Link = RBA_Base_Link + Monetary_Policy_Link + str(Start_Year) + "/"
    
    Response = requests.get(Complete_Link)
    Soup = BeautifulSoup(Response.text,"html.parser")

    UL = Soup.find("ul", {"class": "list-articles"})
    As = UL.find_all("a")
    Minutes_Links = [{"Link" : x["href"], "Title" : x.text.replace(" ", "_")} for x in As]

    ETL_Data(Minutes_Links)
    
    Start_Year += 1